<div style="font-family:Inter; text-align:center; font-size:36px;
            background-color:#ead1dc; border-radius: 20px; padding: 10px">
    <b>Introduction</b></div>

<div style="font-size: 26px; font-family:Inter"><b>Objectives</b></div>
<hr style="height:6px; background-color:#9c9598"></hr>
<p style="font-size: 16px; font-family:Inter">Make an <b>updatable FIFA 23 players dataset</b>, new players will be added and prices will be updated, possibiliting the price analysis. Also, it helps me to improve in <b>web scraping and data cleaning</b>. I hope you enjoy.</p>

<div style="font-size: 26px; font-family:Inter"><b>Technologies and Concepts used</b></div>
<hr style="height:6px; background-color:#9c9598"></hr>
<div style="font-size: 16px; font-family:Inter">
    <ul>
        <li> <b>Beautiful Soup</b> for Web Scraping;</li>   
        <li> <b>URLlib</b> for requests;</li>
        <li> <b>Pandas</b> for data manipulation and data cleaning; </li>
    </ul>
</div>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/ed/Pandas_logo.svg/1200px-Pandas_logo.svg.png" style="width:150px;height:-100px"></img>

<div style="font-size: 26px; font-family:Inter"><b>From where the data was scraped</b></div>
<hr style="height:6px; background-color:#9c9598"></hr>
<p style="font-size: 16px; font-family:Inter"><b>FUTBIN</b> is a FIFA site that contains the FIFA 23 players and features database.</p>

<img src="https://cdn.futbin.com/design/img/futbinBigLogo-w.png" style="background-color:#75ddac;border-radius:20px"><img>

<div style="font-family:Inter; text-align:center; font-size:36px;
            background-color:#ead1dc; border-radius: 20px; padding: 10px">
    <b>Imports</b></div>

In [1]:
# Data Manipulation
import numpy as np
import pandas as pd

# Web Scraping
from urllib.request import Request, urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup

# Time
from datetime import date
from time import sleep

<div style="font-family:Inter; text-align:center; font-size:36px;
            background-color:#ead1dc; border-radius: 20px; padding: 10px">
    <b>Creating a Pipeline Step by Step</b></div>

<div style="font-size: 26px; font-family:Inter"><b>Example - Top 30 Gold Players - Mbappé</b></div>
<hr style="height:6px; background-color:#9c9598"></hr>

In [2]:
url = "https://www.futbin.com/players?page=1&version=gold_rare&pos_type=all"

headers = {'User-Agent': 'Mozilla/5.0'}
req = Request(url, headers=headers)
response = urlopen(req)
html = response.read()


soup = BeautifulSoup(html, 'html.parser')

In [3]:
example = soup.find_all("tr")[4] # Mbappé

In [4]:
attributes = [td.get_text() for td in example.find_all("td")]

<div style="font-size: 26px; font-family:Inter"><b>Clean the Data</b></div>
<hr style="height:6px; background-color:#9c9598"></hr>

In [5]:
def flatten_list(list_):
    output = []
    for sublist in list_:
        output.extend(sublist)
        
    return output

In [6]:
attributes = [i.strip() for i in attributes]

<div style="font-size: 26px; font-family:Inter"><b>Get the Positions, Run Style and Price</b></div>
<hr style="height:6px; background-color:#9c9598"></hr>
<p style="font-size: 16px; font-family:Inter">Get these three attributes by splitting by "\n". This will generate a list of lists, so it flattens to an one dimensional vector.</p>

In [7]:
attributes = flatten_list([i.split("\n") for i in attributes])

<div style="font-size: 26px; font-family:Inter"><b>Get the Workrates</b></div>
<hr style="height:6px; background-color:#9c9598"></hr>
<p style="font-size: 16px; font-family:Inter">Get the attack and defense workrates by splitting by "\\" and then flattening again.</p>

In [8]:
attributes = flatten_list([i.split("\\") for i in attributes])

<div style="font-size: 26px; font-family:Inter"><b>Cleaning again removing empty values</b></div>
<hr style="height:6px; background-color:#9c9598"></hr>
<p style="font-size: 16px; font-family:Inter">After all, there are items that are empty or dirty with "\n" or "\r", so it cleans it and generates a final list result.</p>

In [9]:
attributes = [item.strip() for item in attributes if item.strip() != ""]

<div style="font-size: 26px; font-family:Inter"><b>Partial Result</b></div>
<hr style="height:6px; background-color:#9c9598"></hr>

In [10]:
attributes

['Kylian Mbappé',
 '91',
 'ST',
 'CF,LW',
 'Normal',
 'Controlled',
 '835K',
 '0.60%',
 '5',
 '4',
 'H',
 'L',
 '97',
 '89',
 '80',
 '92',
 '36',
 '76',
 '182cm | 6\'0"',
 'Unique  (73kg)',
 '-1999',
 '470',
 '2223']

<div style="font-size: 26px; font-family:Inter"><b>Get player information</b></div>
<hr style="height:6px; background-color:#9c9598"></hr>
<p style="font-size: 16px; font-family:Inter">Get player information for club, nation and league. They all are links, so they are tagged as "&lt a &gt".</p>

In [11]:
player_information = example.find_all("span")[0].find_all("a")
player_information = [item['data-original-title'] for item in player_information]
player_information

['Paris SG', 'France', 'Ligue 1']

<div style="font-size: 26px; font-family:Inter"><b>Joining all the pipeline</b></div>
<hr style="height:6px; background-color:#9c9598"></hr>
<p style="font-size: 16px; font-family:Inter">Joining all the pipeline in order to create a dictionary of each player, which all values are labeled.</p>

In [12]:
attributes_names = ["Name", "Club", "Nation", "League", "Rating", "Main_Position", "Alternate_Positions", 
                    "Card_Version", "Run_Style", "Price", "Price_Variation", "Skills_Star", "Weak_Foot_Star", 
                    "Attack_Workrate", "Defense_Workrate", 
                    "Pace / Diving", "Shooting / Handling",
                    "Passing / Kicking", "Dribbling / Reflexes", 
                    "Defense / Speed", "Physical / Positioning", 
                    "Height", "BodyType", "Popularity", "Base_Stats", "Ingame_Stats"]

In [13]:
for e, info in enumerate(player_information):
    attributes.insert(e + 1, info)

In [14]:
player_dict = dict(zip(attributes_names, attributes))
player_dict

{'Name': 'Kylian Mbappé',
 'Club': 'Paris SG',
 'Nation': 'France',
 'League': 'Ligue 1',
 'Rating': '91',
 'Main_Position': 'ST',
 'Alternate_Positions': 'CF,LW',
 'Card_Version': 'Normal',
 'Run_Style': 'Controlled',
 'Price': '835K',
 'Price_Variation': '0.60%',
 'Skills_Star': '5',
 'Weak_Foot_Star': '4',
 'Attack_Workrate': 'H',
 'Defense_Workrate': 'L',
 'Pace / Diving': '97',
 'Shooting / Handling': '89',
 'Passing / Kicking': '80',
 'Dribbling / Reflexes': '92',
 'Defense / Speed': '36',
 'Physical / Positioning': '76',
 'Height': '182cm | 6\'0"',
 'BodyType': 'Unique  (73kg)',
 'Popularity': '-1999',
 'Base_Stats': '470',
 'Ingame_Stats': '2223'}

<div style="font-size: 26px; font-family:Inter"><b>Players without Alternate Position problem</b></div>
<hr style="height:6px; background-color:#9c9598"></hr>
<p style="font-size: 16px; font-family:Inter">Players that don't have an alternate position have the value as "", so the function cleans it, implying in missing values. One example is Van Dijk, his values vector has 25 values instead of 26 like Mbappé, so for these players it adds "0" in the alternate position field using the same attributes names dictionary.</p>

In [15]:
van_dijk = soup.find_all("tr")[12]

In [16]:
attributes = [td.get_text() for td in van_dijk.find_all("td")]
attributes = [i.strip() for i in attributes]

attributes = flatten_list([i.split("\n") for i in attributes])
attributes = flatten_list([i.split("\\") for i in attributes])

attributes = [item.strip() for item in attributes if item.strip() != ""]

player_information = van_dijk.find_all("span")[0].find_all("a")
player_information = [item['data-original-title'] for item in player_information]

for e, info in enumerate(player_information):
    attributes.insert(e + 1, info)
    
attributes

['Virgil van Dijk',
 'Liverpool',
 'Netherlands',
 'Premier League',
 '90',
 'CB',
 'Normal',
 'Lengthy',
 '72K',
 '2.70%',
 '2',
 '3',
 'M',
 'H',
 '81',
 '60',
 '71',
 '72',
 '91',
 '86',
 '193cm | 6\'4"',
 'Unique  (92kg)',
 '2933',
 '461',
 '2149']

In [17]:
len(attributes)

25

<div style="font-size: 26px; font-family:Inter"><b>Joining all the steps into a function</b></div>
<hr style="height:6px; background-color:#9c9598"></hr>

In [18]:
def get_player_attributes(row):
    attributes = [td.get_text() for td in row.find_all("td")]
    attributes = [i.strip() for i in attributes]
    
    attributes = flatten_list([i.split("\n") for i in attributes])
    attributes = flatten_list([i.split("\\") for i in attributes])
    
    attributes = [item.strip() for item in attributes if item.strip() != ""]
    
    # Club, League and Nation
    player_information = row.find_all("span")[0].find_all("a")
    player_information = [item['data-original-title'] for item in player_information]
    
    # Adding to the attributes list
    for e, info in enumerate(player_information):
        attributes.insert(e + 1, info)
        
    # If a player doesn't have an alternate position
    if len(attributes) == 25:
        attributes.insert(6, None)
    
    player_dict = dict(zip(attributes_names, attributes))
    return player_dict

<div style="font-size: 26px; font-family:Inter"><b>Example with the top 30 gold players</b></div>
<hr style="height:6px; background-color:#9c9598"></hr>

In [19]:
gold_players_page = soup.find_all("tr")[::2]

In [20]:
top30_gold_players = list()

for player_row in gold_players_page:
    player_attributes = get_player_attributes(player_row)
    top30_gold_players.append(player_attributes)

In [21]:
top30_gold_players = pd.DataFrame(top30_gold_players)
top30_gold_players.head()

,Name,Club,Nation,League,Rating,Main_Position,Alternate_Positions,Card_Version,Run_Style,Price,...,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Height,BodyType,Popularity,Base_Stats,Ingame_Stats
0,Karim Benzema,Real Madrid,France,LaLiga Santander,91,CF,ST,Normal,Controlled,66.5K,...,88,83,87,39,78,"185cm | 6'1""",Average (81kg),3557,455,2196
1,Robert Lewandowski,FC Barcelona,Poland,LaLiga Santander,91,ST,CF,Normal,Controlled,68.5K,...,91,79,86,44,83,"185cm | 6'1""",Unique (81kg),1108,458,2242
2,Kylian Mbappé,Paris SG,France,Ligue 1,91,ST,"CF,LW",Normal,Controlled,835K,...,89,80,92,36,76,"182cm | 6'0""",Unique (73kg),-1999,470,2223
3,Kevin De Bruyne,Manchester City,Belgium,Premier League,91,CM,CAM,Normal,Controlled,67K,...,88,93,87,64,77,"181cm | 5'11""",Unique (70kg),1873,483,2336
4,Lionel Messi,Paris SG,Argentina,Ligue 1,91,RW,RM,Normal,Explosive,70K,...,89,90,94,34,64,"169cm | 5'7""",Messi (67kg),2766,452,2232


<div style="font-family:Inter; text-align:center; font-size:36px;
            background-color:#ead1dc; border-radius: 20px; padding: 10px">
    <b>Missing Values</b></div>

<div style="font-size: 26px; font-family:Inter"><b>Casillas Problem</b></div>
<hr style="height:6px; background-color:#9c9598"></hr>
<p style="font-size: 16px; font-family:Inter">For example, Casillas doesn't have height neither bodytype, besides the alternate position, like a few players as Cafu, that doesn't have the bodytype.</p> 
<p style="font-size: 16px; font-family:Inter">The solution for the problem is to find what differentiates the height and bodytype from the other attributes. The height has its separator "|" (differs cm to feet) while bodytype contains the weight of the player in "()".</p>
<p style="font-size: 16px; font-family:Inter">So if a player doens't have the separator "|" in any of his attributes, it means that he doenst' have height, so it completes with "0" or "Other". The same way for bodytype.</p>

In [22]:
page = "https://www.futbin.com/players?page=3&pos_type=all"

headers = {'User-Agent': 'Mozilla/5.0'}
req = Request(page, headers=headers)
response = urlopen(req)
html = response.read()


soup_p = BeautifulSoup(html, 'html.parser')

In [23]:
casillas = soup_p.find_all("tr")[32]

In [24]:
c_attributes = [td.get_text() for td in casillas.find_all("td")]
c_attributes = [i.strip() for i in c_attributes]

c_attributes = flatten_list([i.split("\n") for i in c_attributes])
c_attributes = flatten_list([i.split("\\") for i in c_attributes])
c_attributes = [i.strip() for i in c_attributes if i.strip() != ""]
c_attributes

['Casillas Fernández',
 '92',
 'GK',
 'Icon',
 'Controlled',
 '489K',
 '2.09%',
 '1',
 '3',
 'M',
 'M',
 '93',
 '89',
 '85',
 '95',
 '60',
 '91',
 '76',
 '513',
 '1355']

<div style="font-size: 26px; font-family:Inter"><b>Function to find these missing values</b></div>
<hr style="height:6px; background-color:#9c9598"></hr>
<p style="font-size: 16px; font-family:Inter"> It iterates for the attributes list and returns False if doesn't contain. If the function find the separator in an item it will break the loop and the value will keep his True value, meaning that it found and the player has the attribute.</p>

In [25]:
def find_missing_att(list_att, sep):
    '''Iterates for the attributes list and  returns False if doesn't contain.
    If the function find the separator in an item it will break the loop and 
    the value will keep his True value, meaning that it found and the player has
    the attribute.'''
    for att in list_att:
        value = sep in att
        if value:
            break
            
    return value

<div style="font-size: 26px; font-family:Inter"><b>Searching for height using "|"</b></div>
<hr style="height:6px; background-color:#9c9598"></hr>

In [26]:
# Searching for height using "|"
print("Casillas (don't have): ", find_missing_att(c_attributes, "|"))
print("Van Dijk (have): ", find_missing_att(attributes, "|"))

Casillas (don't have):  False
Van Dijk (have):  True


<div style="font-size: 26px; font-family:Inter"><b>Searching for bodytype / weight using "("</b></div>
<hr style="height:6px; background-color:#9c9598"></hr>

In [27]:
# Searching for bodytype / weight using "("
print("Casillas (don't have): ", find_missing_att(c_attributes, "("))
print("Van Dijk (have): ", find_missing_att(attributes, "("))

Casillas (don't have):  False
Van Dijk (have):  True


<div style="font-size: 26px; font-family:Inter"><b>Update the searching function</b></div>
<hr style="height:6px; background-color:#9c9598"></hr>

In [28]:
def get_player_attributes(row):
    attributes = [td.get_text() for td in row.find_all("td")]
    attributes = [i.strip() for i in attributes]
    
    attributes = flatten_list([i.split("\n") for i in attributes])
    attributes = flatten_list([i.split("\\") for i in attributes])
    
    attributes = [item.strip() for item in attributes if item.strip() != ""]
    
    # Club, League and Nation
    player_information = row.find_all("span")[0].find_all("a")
    player_information = [item['data-original-title'] for item in player_information]
    
    # Adding to the attributes list
    for e, info in enumerate(player_information):
        attributes.insert(e + 1, info)
    
    # Searching for missing values
    ## Height
    if not find_missing_att(attributes, "|"):
        attributes.insert(-3, "0")
    ## BodyType -- completed with the most common
    if not find_missing_att(attributes, "("):
        attributes.insert(-3, "Normal")
    ## Price Variation:
    if not find_missing_att(attributes, "%"):
        attributes.insert(10, "0")
        
    # Player doesn't have card version labeled
    runstyle = ["Controlled", "Explosive", "Lengthy"]
    if len(attributes) < 25:
        for style in runstyle:
            if style in attributes:
                index = attributes.index(style)
                attributes.insert(index - 1, "Normal")
    
    # If a player doesn't have an alternate position
    if len(attributes) == 25:
        attributes.insert(6, "0")
    
    player_dict = dict(zip(attributes_names, attributes))
    return player_dict

In [29]:
get_player_attributes(casillas)

{'Name': 'Casillas Fernández',
 'Club': 'FUT ICONS',
 'Nation': 'Spain',
 'League': 'Icons',
 'Rating': '92',
 'Main_Position': 'GK',
 'Alternate_Positions': '0',
 'Card_Version': 'Icon',
 'Run_Style': 'Controlled',
 'Price': '489K',
 'Price_Variation': '2.09%',
 'Skills_Star': '1',
 'Weak_Foot_Star': '3',
 'Attack_Workrate': 'M',
 'Defense_Workrate': 'M',
 'Pace / Diving': '93',
 'Shooting / Handling': '89',
 'Passing / Kicking': '85',
 'Dribbling / Reflexes': '95',
 'Defense / Speed': '60',
 'Physical / Positioning': '91',
 'Height': '0',
 'BodyType': 'Normal',
 'Popularity': '76',
 'Base_Stats': '513',
 'Ingame_Stats': '1355'}

<div style="font-family:Inter; text-align:center; font-size:36px;
            background-color:#ead1dc; border-radius: 20px; padding: 10px">
    <b>Pipeline to iterate through pages</b></div>

In [30]:
def authentication_page(page):
    headers = {'User-Agent': 'Mozilla/5.0'}
    req = Request(page, headers=headers)
    response = urlopen(req)
    html = response.read()
    soup_a = BeautifulSoup(html, 'html.parser')
    return soup_a

In [31]:
def iterate_for_page(index):
    page_url = "https://www.futbin.com/players?page=" + str(index)
    
    # Authentication
    soup_p = authentication_page(page_url)
    
    # Getting the players from the page
    page_players = soup_p.find_all("tr")[::2]
    page_dataframe = list()
    
    # Iteration
    for player_row in page_players:
        player_attributes = get_player_attributes(player_row)
        page_dataframe.append(player_attributes)
        
    page_dataframe = pd.DataFrame(page_dataframe)
    return page_dataframe

<div style="font-size: 26px; font-family:Inter"><b>Test until page 10</b></div>
<hr style="height:6px; background-color:#9c9598"></hr>

In [32]:
# Test until page 10
players_till_10 = pd.DataFrame([])
for i in range(10):
    page_df = iterate_for_page(i + 1)
    players_till_10 = pd.concat([players_till_10, page_df])

In [33]:
display(players_till_10.head())
display(players_till_10.tail())

,Name,Club,Nation,League,Rating,Main_Position,Alternate_Positions,Card_Version,Run_Style,Price,...,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Height,BodyType,Popularity,Base_Stats,Ingame_Stats
0,Pelé,FUT ICONS,Brazil,Icons,98,CAM,"CF,ST",Icon,Explosive,4.85M,...,96,93,96,60,76,"173cm | 5'8""",Unique (70kg),2925,516,2513
1,Ronaldo,FUT ICONS,Brazil,Icons,96,ST,CF,Icon,Controlled,9.7M,...,95,81,95,45,76,"183cm | 6'0""",R9 (78kg),3170,489,2341
2,Zinedine Zidane,FUT ICONS,France,Icons,96,CAM,CM,Icon,Controlled,2.94M,...,92,96,95,75,86,"185cm | 6'1""",Average (77kg),1175,529,2548
3,Pelé,FUT ICONS,Brazil,Icons,96,CF,"CAM,ST",World Cup ICON,Explosive,3.9M,...,94,92,95,58,75,"173cm | 5'8""",Unique (70kg),914,510,2475
4,Pelé,FUT ICONS,Brazil,Icons,95,CF,"CAM,ST",Icon,Explosive,3.1M,...,93,90,95,56,75,"173cm | 5'8""",Unique (69kg),1763,505,2452


,Name,Club,Nation,League,Rating,Main_Position,Alternate_Positions,Card_Version,Run_Style,Price,...,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Height,BodyType,Popularity,Base_Stats,Ingame_Stats
25,Ederson,Manchester City,Brazil,Premier League,89,GK,0,Normal,Lengthy,38K,...,82,93,88,64,88,"188cm | 6'2""",Unique (86kg),399,502,1215
26,N'Golo Kanté,Chelsea,France,Premier League,89,CDM,CM,Normal,Controlled,36.5K,...,66,74,81,87,82,"168cm | 5'6""",Unique (70kg),1551,462,2184
27,Jan Oblak,Atlético de Madrid,Slovenia,LaLiga Santander,89,GK,0,Normal,Controlled,36.75K,...,90,78,89,49,87,"188cm | 6'2""",Unique (87kg),142,479,1040
28,Morientes,HERO,Spain,LaLiga Santander,89,ST,CF,Hero,Controlled,38K,...,90,77,83,45,86,"186cm | 6'1""",High & Average (79kg),1162,465,2242
29,Abedi Pelé,HERO,Ghana,Ligue 1,89,CAM,"LM,CF",Hero,Explosive,204K,...,83,87,91,48,66,"174cm | 5'9""",Average (67kg),694,466,2267


<div style="font-family:Inter; text-align:center; font-size:36px;
            background-color:#ead1dc; border-radius: 20px; padding: 10px">
    <b>Scraping all players</b></div>

<div style="font-size: 26px; font-family:Inter"><b>Iteration through all pages - 608 in total</b></div>
<hr style="height:6px; background-color:#9c9598"></hr>
<p style="font-size: 16px; font-family:Inter"> <b>Care taken:</b> in order to get all the players, it had to iterate in each page, requesting to scrape. If the iteration is too fast, it gives the <b style="color:#cc0000; background-color:#f5abab">HTTPError 429: Too Many Requests</b>. So, every 5 iterations it sleeps 15 seconds, it takes more time but gives no error.</p>

In [34]:
all_players = pd.DataFrame([])

In [35]:
for i in range(608):
    try:
        page_df = iterate_for_page(i + 1)
        all_players = pd.concat([all_players, page_df])
        
        if (i + 1) % 5 == 0:
            sleep(15)
        if (i + 1) % 50 == 0:
            print(f"Page {i + 1} Successful")
            
    except HTTPError as err:
        print("--Erro--", err, sep = "\n")
        sleep(30)

Page 50 Successful
Page 100 Successful
Page 150 Successful
Page 200 Successful
Page 250 Successful
Page 300 Successful
Page 350 Successful
Page 400 Successful
Page 450 Successful
Page 500 Successful
Page 550 Successful
Page 600 Successful


In [36]:
all_players.head()

,Name,Club,Nation,League,Rating,Main_Position,Alternate_Positions,Card_Version,Run_Style,Price,...,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Height,BodyType,Popularity,Base_Stats,Ingame_Stats
0,Pelé,FUT ICONS,Brazil,Icons,98,CAM,"CF,ST",Icon,Explosive,4.85M,...,96,93,96,60,76,"173cm | 5'8""",Unique (70kg),2925,516,2513
1,Ronaldo,FUT ICONS,Brazil,Icons,96,ST,CF,Icon,Controlled,9.7M,...,95,81,95,45,76,"183cm | 6'0""",R9 (78kg),3170,489,2341
2,Zinedine Zidane,FUT ICONS,France,Icons,96,CAM,CM,Icon,Controlled,2.94M,...,92,96,95,75,86,"185cm | 6'1""",Average (77kg),1175,529,2548
3,Pelé,FUT ICONS,Brazil,Icons,96,CF,"CAM,ST",World Cup ICON,Explosive,3.9M,...,94,92,95,58,75,"173cm | 5'8""",Unique (70kg),914,510,2475
4,Pelé,FUT ICONS,Brazil,Icons,95,CF,"CAM,ST",Icon,Explosive,3.1M,...,93,90,95,56,75,"173cm | 5'8""",Unique (69kg),1763,505,2452


<div style="font-family:Inter; text-align:center; font-size:36px;
            background-color:#ead1dc; border-radius: 20px; padding: 10px">
    <b>Processing the Dataset</b></div>

<div style="font-size: 26px; font-family:Inter"><b>Transforming columns in numeric</b></div>
<hr style="height:6px; background-color:#9c9598"></hr>
<div style="font-size: 16px; font-family:Inter"> There are columns that could be transformed in numeric:
    <ul>
        <li> Rating;</li>   
        <li>Skills Star;</li>
        <li>Weakfoot Star;</li>
        <li>Pace / Diving;</li>
        <li>Shooting / Handling;</li>
        <li>Passing / Kicking;</li>
        <li>Dribbling / Reflexes;</li>
        <li>Defense / Speed;</li>
        <li>Physical / Positioning;</li>
        <li>Popularity;</li>
        <li>Base Stats;</li>
        <li>Ingame Stats;</li>
    </ul>
</div>

In [37]:
all_players['Rating'] = pd.to_numeric(all_players.Rating)
all_players['Skills_Star'] = pd.to_numeric(all_players.Skills_Star)
all_players['Weak_Foot_Star'] = pd.to_numeric(all_players.Weak_Foot_Star)

In [38]:
all_players['Pace / Diving'] = pd.to_numeric(all_players['Pace / Diving'], errors="coerce") 
all_players['Shooting / Handling'] = pd.to_numeric(all_players['Shooting / Handling'])
all_players['Passing / Kicking'] = pd.to_numeric(all_players['Passing / Kicking'])
all_players['Dribbling / Reflexes'] = pd.to_numeric(all_players['Dribbling / Reflexes'])
all_players['Defense / Speed'] = pd.to_numeric(all_players['Defense / Speed'])
all_players['Physical / Positioning'] = pd.to_numeric(all_players['Physical / Positioning'])

In [39]:
all_players['Popularity'] = pd.to_numeric(all_players.Popularity)
all_players['Base_Stats'] = pd.to_numeric(all_players.Base_Stats)
all_players['Ingame_Stats'] = pd.to_numeric(all_players.Ingame_Stats)

<div style="font-size: 26px; font-family:Inter"><b>Price Processing</b></div>
<hr style="height:6px; background-color:#9c9598"></hr>
<p style="font-size: 16px; font-family:Inter"> The price column has a different composition, with "K" meaning 1000 and "M" Million.</p>

In [40]:
price_processing = all_players['Price']

In [41]:
symbols = ("M", "K")
prices_p = list()
for price in price_processing:
    if symbols[0] in price:
        price_p = price[:-1]
        price_p = round(float(price_p) * 1e6, 2)
    elif symbols[1] in price:
        price_p = price[:-1]
        price_p = round(float(price_p) * 1e3, 2)
    else:
        price_p = price
        
    prices_p.append(price_p)

In [42]:
all_players['Price'] = prices_p

<div style="font-size: 26px; font-family:Inter"><b>Fixing the Index and Droping Null Values</b></div>
<hr style="height:6px; background-color:#9c9598"></hr>
<p style="font-size: 16px; font-family:Inter"> The index is duplicate - a number between 0 and 29 - so it has to be fixed. Also, there are a few players that are the exception in missing values, so it's more efficient to drop these rows than creating a specific rule in the searching function.</p>

In [43]:
# Fixing the index
all_players = all_players.reset_index().drop('index', axis=1)
all_players.head()

,Name,Club,Nation,League,Rating,Main_Position,Alternate_Positions,Card_Version,Run_Style,Price,...,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Height,BodyType,Popularity,Base_Stats,Ingame_Stats
0,Pelé,FUT ICONS,Brazil,Icons,98,CAM,"CF,ST",Icon,Explosive,4850000.0,...,96,93,96,60,76,"173cm | 5'8""",Unique (70kg),2925,516,2513
1,Ronaldo,FUT ICONS,Brazil,Icons,96,ST,CF,Icon,Controlled,9700000.0,...,95,81,95,45,76,"183cm | 6'0""",R9 (78kg),3170,489,2341
2,Zinedine Zidane,FUT ICONS,France,Icons,96,CAM,CM,Icon,Controlled,2940000.0,...,92,96,95,75,86,"185cm | 6'1""",Average (77kg),1175,529,2548
3,Pelé,FUT ICONS,Brazil,Icons,96,CF,"CAM,ST",World Cup ICON,Explosive,3900000.0,...,94,92,95,58,75,"173cm | 5'8""",Unique (70kg),914,510,2475
4,Pelé,FUT ICONS,Brazil,Icons,95,CF,"CAM,ST",Icon,Explosive,3100000.0,...,93,90,95,56,75,"173cm | 5'8""",Unique (69kg),1763,505,2452


In [44]:
# Viewing the players with these null values
na_players = all_players[all_players["Pace / Diving"].isna()]
na_players

,Name,Club,Nation,League,Rating,Main_Position,Alternate_Positions,Card_Version,Run_Style,Price,...,Shooting / Handling,Passing / Kicking,Dribbling / Reflexes,Defense / Speed,Physical / Positioning,Height,BodyType,Popularity,Base_Stats,Ingame_Stats
9666,Rolando Aarons,Motherwell,Jamaica,Scottish Premiership (SPFL),67,LW,0,"LWB,LM,RW",Normal,Explosive,...,78,62,66,70,39,52,"175cm | 5'9""",0,367,1709
9904,Rolando Aarons,Huddersfield,Jamaica,EFL Championship (ENG 2),67,LW,0,"LWB,LM,RW",non-rare,Explosive,...,78,62,66,70,39,52,"175cm | 5'9""",0,367,1709
12640,Harry Smith,Exeter City,England,EFL League One (ENG 3),64,ST,0,CF,Normal,Controlled,...,53,64,48,56,31,78,"193cm | 6'4""",4,330,1530
12959,Harry Smith,Leyton Orient,England,EFL League Two (ENG 4),64,ST,0,CF,Rare,Controlled,...,53,64,48,56,31,78,"193cm | 6'4""",-1,330,1530
17746,Frimpong,AFC Wimbledon,England,EFL League Two (ENG 4),54,CDM,0,Normal,non-rare,Controlled,...,65,37,45,54,52,61,0,0,314,1463
17909,Ridley,Sutton United,England,EFL League Two (ENG 4),53,CB,0,Normal,Rare,Controlled,...,64,23,27,33,56,58,0,0,261,1204
18227,Charles-Cook,Sutton United,England,EFL League Two (ENG 4),50,RB,0,RWB,non-rare,Controlled,...,65,24,31,43,49,58,0,-1,270,1231


In [45]:
all_players.drop(na_players.index, inplace=True, errors='ignore')
all_players = all_players.reset_index().drop('index', axis=1)

In [46]:
all_players['Price'] = pd.to_numeric(all_players['Price'])

In [47]:
all_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18233 entries, 0 to 18232
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Name                    18233 non-null  object 
 1   Club                    18233 non-null  object 
 2   Nation                  18233 non-null  object 
 3   League                  18233 non-null  object 
 4   Rating                  18233 non-null  int64  
 5   Main_Position           18233 non-null  object 
 6   Alternate_Positions     18233 non-null  object 
 7   Card_Version            18233 non-null  object 
 8   Run_Style               18233 non-null  object 
 9   Price                   18233 non-null  float64
 10  Price_Variation         18233 non-null  object 
 11  Skills_Star             18233 non-null  int64  
 12  Weak_Foot_Star          18233 non-null  int64  
 13  Attack_Workrate         18233 non-null  object 
 14  Defense_Workrate        18233 non-null

<div style="font-family:Inter; text-align:center; font-size:36px;
            background-color:#ead1dc; border-radius: 20px; padding: 10px">
    <b>Exporting as csv</b></div>

In [48]:
path = "fifa23_players_"
today = str(date.today())
path += today + ".csv"

all_players.to_csv(path, index=False)